# Zebrafish Trajectory Analysis Script
This script analyzes the movement of zebrafish in a 12-well plate experiment. It reads trajectory data, preprocesses it, calculates trajectories and velocities, performs statistical analysis, and generates visualizations.
## 1. Imports and Constants
This section imports necessary libraries and defines constants for the experimental setup.
**Explanation:**
*   **Libraries:** Imports the pandas library for data manipulation, matplotlib for plotting, numpy for numerical calculations, seaborn for statistical visualizations, and scipy.stats for statistical tests.
*   **Constants:** Defines key parameters such as frames per second (`FPS`), tank dimensions (`TANK_WIDTH_CM`, `TANK_HEIGHT_CM`), video resolution (`RESOLUTION_WIDTH`, `RESOLUTION_HEIGHT`), number of fish (`NUM_FISH`), image DPI (`DPI`).
*   **Fish Name Mapping:** Defines a dictionary `fish_name_mapping` to assign names to each fish, including duplicates.


In [172]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import scipy.stats as stats

# Constants
FPS = 25
TANK_WIDTH_CM = 8.5 # in cm
TANK_HEIGHT_CM = 12.776 # in cm
RESOLUTION_WIDTH = 1360 # in pixels
RESOLUTION_HEIGHT = 960 # in pixels
NUM_FISH = 18
DPI = 600  # for saving figures

# Fish name mapping
fish_name_mapping = {
    1: 'Control',
    2: 'Control',
    3: '0.08 mg/mL',
    4: '0.08 mg/mL',
    5: '0.16 mg/mL',
    6: '0.16mg/mL',
    7: '0.24 mg/mL',
    8: '0.24 mg/mL',
    9: '0.32 mg/mL',
    10: '0.32 mg/mL',
    11: '3.6 mg/mL',
    12: '3.6 mg/mL',
    13: '4.4 mg/mL',
    14: '4.4 mg/mL',
    15: '5.2 mg/mL',
    16: '5.2 mg/mL',
    17: '6 mg/mL',
    18: '6 mg/mL'
   
}

## 2. Data Loading and Preprocessing

This section loads the trajectory data from a CSV file and preprocesses it.
**Explanation:**

*   **Load Data:** Reads the `trajectories.csv` file into a pandas DataFrame named `df`.
*   **Interpolation:** Fills missing data points (NaN) in the DataFrame using linear interpolation with a polynomial of order 3.  This smooths the data.
*   **Time Column:** Creates a new column named `time` representing the time in seconds for each frame based on the DataFrame index and the frame rate (`FPS`).

In [173]:
# Load the data
df = pd.read_excel(r"C:\Users\Lenovo\Desktop\ishika\ishika\trajectories.xlsx")

# Data Preprocessing
df.interpolate(method='linear', inplace=True)  # interpolate values
df['time'] = df.index / FPS  # Add a time column in seconds


## 3. Pixel to Centimeter Conversion

This section defines a function to convert pixel coordinates to real-world centimeters.

**Explanation:**

*   **`pixel_to_cm(x, y)` Function:**  Takes `x` and `y` pixel coordinates as input and returns the corresponding centimeter coordinates based on the tank's dimensions and video resolution.
*   **Coordinate Conversion:** The `pixel_to_cm` function is applied to each x, y coordinate pair (e.g., 'x1', 'y1', 'x2', 'y2', etc.) in the DataFrame using the `.apply` method. New columns are generated, such as `x1_cm`, `y1_cm` and so on to represent the positions in centimeters.

In [174]:
# Function to convert pixel coordinates to centimeters
def pixel_to_cm(x, y):
    x_cm = (x / RESOLUTION_WIDTH) * TANK_WIDTH_CM
    y_cm = (y / RESOLUTION_HEIGHT) * TANK_HEIGHT_CM
    return x_cm, y_cm

# %%
# Convert all coordinates to centimeters and add as new columns
for i in range(1, NUM_FISH + 1):
    df[f'x{i}_cm'], df[f'y{i}_cm'] = zip(*df.apply(lambda row: pixel_to_cm(row[f'x{i}'], row[f'y{i}']), axis=1))


## 4. Defining Wells and Normalizing Trajectories

This section defines the well locations and removes data outside the wells.

**Explanation:**

*   **Well Centers and Radius:** Defines a dictionary `well_centers` to hold the (x, y) coordinates of the centers of each well, and `well_radius` represents the pixel radius of a well.
*   **`is_inside_well` Function:**  Takes a point (x, y), a well center, and a radius as inputs. It returns `True` if the point is inside the circle defined by the well, `False` otherwise.
*   **Data Normalization:**
    *   A copy of the original DataFrame is made and called `df_normalized`.
    *   It then iterates through the fish and apply the `is_inside_well` function using the respective well center. If a position of a fish is located outside of its well, the x, y coordinates in `df_normalized` are converted to `NaN`, effectively removing them.

In [175]:
# Define the approximate well centers and radius
well_centers = {
1: (997, 602), 2: (1002, 821), 3: (124, 179), 4: (122, 394), 5: (774, 167), 6: (777, 386),
7: (780, 606), 8: (780, 826), 9: (994, 166), 10: (995, 382), 11: (122, 612), 12: (130, 829),
13: (345, 612), 14: (349, 829), 15: (557, 176), 16: (557, 391), 17: (560, 609), 18: (567, 827)

}
well_radius = 36.5  # Adjust based on actual pixel size

In [176]:
# Function to check if a point is inside a circular well
def is_inside_well(x, y, well_center, well_radius):
    xc, yc = well_center
    return (x - xc)**2 + (y - yc)**2 <= well_radius**2

In [177]:
# Create a copy of the dataframe, and remove all the points outside the designated well.
df_normalized = df.copy()

In [178]:
# Iterate over all fish and mark points outside their respective well as NaN
for i in range(1, NUM_FISH + 1):
    x_col = f'x{i}'
    y_col = f'y{i}'
    
    well_center = well_centers[i]
    
    # Apply function to detect outliers and set them to NaN
    mask_outside = ~df_normalized.apply(lambda row: is_inside_well(row[x_col], row[y_col], well_center, well_radius), axis=1)
    df_normalized.loc[mask_outside, [x_col, y_col]] = np.nan


## 5. Analysis Function

This section defines a function to analyze the zebrafish trajectories within a given time range.
**Explanation:**

*   **`analyze_zebrafish(start_time, end_time)` Function:**  The core analysis logic is encapsulated within this function. It takes `start_time` and `end_time` parameters to control the analysis window.
*   **Combined Trajectory Plot:** Creates a plot with trajectories of all fish in one plot, with labels from the `fish_name_mapping`.
*   **Separate Density Plots:** Each fish now has its own separate density plot, and the filenames are saved using the `fish_name_mapping` and an identifier `i`.
*    **Statistical Tests:** Performs an ANOVA and Kruskal-Wallis test using all of the data points in the selected range.
*   **CSV Saving:** The results of all statistical tests, distances, and speed data are saved to distinct CSV files using Pandas DataFrames and the specified naming convention.

In [179]:
# Print to check which names were actually used.
print(f"Using the following names: {fish_name_mapping}")

Using the following names: {1: 'Control', 2: 'Control', 3: '0.08 mg/mL', 4: '0.08 mg/mL', 5: '0.16 mg/mL', 6: '0.16mg/mL', 7: '0.24 mg/mL', 8: '0.24 mg/mL', 9: '0.32 mg/mL', 10: '0.32 mg/mL', 11: '3.6 mg/mL', 12: '3.6 mg/mL', 13: '4.4 mg/mL', 14: '4.4 mg/mL', 15: '5.2 mg/mL', 16: '5.2 mg/mL', 17: '6 mg/mL', 18: '6 mg/mL'}


In [180]:
def analyze_zebrafish(start_time, end_time):
    # Calculate time range for analysis
    TIME_RANGE_START = start_time
    TIME_RANGE_END = end_time

    print(f"Analyzing data from {TIME_RANGE_START}s to {TIME_RANGE_END}s...")
    # Filter data for specified time range
    df_time_range = df[(df['time'] >= TIME_RANGE_START) & (df['time'] <= TIME_RANGE_END)].copy()

    # 1. Combined Trajectory Plot
    print("Generating combined trajectory plot...")
    plt.figure(figsize=(6, 4))
    for i in range(1, NUM_FISH + 1):
        x_col = f'x{i}_cm'
        y_col = f'y{i}_cm'
        
        # Ensure the columns exist and are not all NaN
        if x_col in df_time_range and y_col in df_time_range and not df_time_range[x_col].isnull().all() and not df_time_range[y_col].isnull().all():
            plt.plot(df_time_range[x_col], df_time_range[y_col], label=fish_name_mapping[i], alpha=0.7)
    plt.title(f'Trajectories of All Fish ({TIME_RANGE_START}s to {TIME_RANGE_END}s)')
    #plt.xlabel('X Coordinates (cm)')
    #plt.ylabel('Y Coordinates (cm)')
    # plot legend outside the plot
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1), fontsize='small')
    # turn of the tics
    plt.xticks([])
    plt.yticks([])
    plt.xlim(0, TANK_WIDTH_CM)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'trajectories_{TIME_RANGE_START}_{TIME_RANGE_END}.png', dpi=DPI)
    plt.close()  # Close the figure to free memory 

    # 2. Individual Density Plots
    print("Generating individual density plots...")
    for i in range(1, NUM_FISH + 1):
        # Use the object-oriented approach for clarity and safety
        fig, ax = plt.subplots(figsize=(5, 5))
        x_col = f'x{i}'
        y_col = f'y{i}'

        fish_name = fish_name_mapping[i]
        safe_fish_name = fish_name.replace('/', '_')

        # Pass the 'ax' object to seaborn to plot on the specific subplot
        sns.kdeplot(x=df_normalized[x_col].dropna(), y=df_normalized[y_col].dropna(),
                    cmap='viridis', fill=True, levels=5, alpha=0.8, ax=ax)

        ax.set_title(f'Density Plot for {fish_name}')
        ax.set_xlabel('X Coordinates (pixels)')
        ax.set_ylabel('Y Coordinates (pixels)')

        plt.savefig(f'density_{safe_fish_name}_{i}_{TIME_RANGE_START}_{TIME_RANGE_END}.png', dpi=DPI)
        plt.close(fig)  # Close the figure to free memory after saving

    # 3. Distance Travelled Bar Plot
    print("Generating distance travelled bar plot...")
    distances = []
    labels_with_ids = []  # To store fish names with IDs
    for i in range(1, NUM_FISH + 1):
        x_col = f'x{i}_cm'
        y_col = f'y{i}_cm'

        # Ensure the columns exist and are not all NaN
        if x_col in df_time_range and y_col in df_time_range and not df_time_range[x_col].isnull().all() and not df_time_range[y_col].isnull().all():
            # Calculate distance travelled by each fish
            distance = np.sqrt((df_time_range[x_col].diff()**2 + df_time_range[y_col].diff()**2)).sum()
            distances.append(distance)
            labels_with_ids.append(f"{fish_name_mapping[i]} ({i})")
            print(f"Distance travelled by {fish_name_mapping[i]} (Fish {i}): {distance:.2f} cm")
            # Add the distance as a new column in the dataframe
            df_time_range[f'distance{i}'] = np.sqrt((df_time_range[x_col].diff()**2 + df_time_range[y_col].diff()**2)).cumsum()
        else:
            print(f"Skipping Fish {i} ({fish_name_mapping[i]}): No valid data available.")
    # Create a bar plot for distances
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(labels_with_ids, distances, color='skyblue')
    ax.set_title(f'Distance Travelled by Each Fish ({TIME_RANGE_START}s to {TIME_RANGE_END}s)', fontsize=16, fontweight='bold')
    ax.set_xlabel('Fish', fontsize=14, fontweight='bold')
    ax.set_ylabel('Distance (cm)', fontsize=14, fontweight='bold')
    ax.tick_params(axis='x', rotation=45, labelsize=12)
    ax.tick_params(axis='y', labelsize=12)
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig(f'distance_travelled_{TIME_RANGE_START}_{TIME_RANGE_END}.png', dpi=DPI, bbox_inches='tight')
    plt.close(fig)  # Close the figure to free memory
  


    # 4. Individual Speed Plots
    print("Generating individual speed plots...")
    # First, calculate all speed columns
    for i in range(1, NUM_FISH + 1):
      df_time_range[f'speed{i}'] = np.sqrt(df_time_range[f'x{i}_cm'].diff()**2 + df_time_range[f'y{i}_cm'].diff()**2) / \
                                       df_time_range['time'].diff()

    # Then, create a plot for each fish
    for i in range(1, NUM_FISH + 1):
        fig, ax = plt.subplots(figsize=(8, 6))

        fish_name = fish_name_mapping[i]
        safe_fish_name = fish_name.replace('/', '_')

        ax.plot(df_time_range['time'], df_time_range[f'speed{i}'], label=fish_name)
        ax.set_xlabel('Time (s)')
        ax.set_ylabel('Speed (cm/s)')
        ax.set_title(f'Speed of {fish_name}')
        ax.grid(True)
        plt.savefig(f'speed_{safe_fish_name}_{i}_{TIME_RANGE_START}_{TIME_RANGE_END}.png', dpi=DPI)
        plt.close(fig)  # Close the figure to free memory

    # 5. Combined Velocity Plots (Violin and Box)
    print("Generating combined velocity plots...")
    df_speed = df_time_range[[f'speed{i}' for i in range(1, NUM_FISH+1)]].copy()
    # Create a mapping of column names to fish names and to their numbers
    speed_col_to_name = {f'speed{i}': f'{fish_name_mapping[i]} {i}' for i in range(1, NUM_FISH + 1)}

    # Rename the columns with both the name and the number.
    df_speed.rename(columns = speed_col_to_name, inplace = True)

    df_melt = df_speed.melt(var_name='Fish', value_name='Velocity')

    # Add the full label
    df_melt['Fish_label'] = df_melt['Fish']

    # --- Violin Plot (Zoomed In) ---
    plt.rcParams['font.family'] = 'Times New Roman'
    fig, ax = plt.subplots(figsize=(12, 8))
    sns.violinplot(x='Fish_label', y='Velocity', data=df_melt, palette='Paired', hue='Fish_label', legend=False, ax=ax)
    ax.tick_params(axis='x', labelsize=16, rotation=45)
    ax.tick_params(axis='y', labelsize=16)
    ax.set_title(f'Velocity Distribution (Violin) of All Fish ({TIME_RANGE_START}s to {TIME_RANGE_END}s)', fontsize=24, fontweight='bold')
    ax.set_xlabel('Fish', fontsize=20, fontweight='bold')
    ax.set_ylabel('Speed (cm/s)', fontsize=20, fontweight='bold')
    ax.grid(False)
    #ax.set_ylim(0, 2)  # Zoom in on the bulk of the data, excluding extreme outliers. Adjust as needed.
    plt.savefig(f'violin_velocity_all_{TIME_RANGE_START}_{TIME_RANGE_END}.png', dpi=DPI, bbox_inches='tight')
    plt.close(fig) # Close the figure to free memory

    # --- Box Plot (Full Range) ---
    fig, ax = plt.subplots(figsize=(12, 8))
    sns.boxplot(x='Fish_label', y='Velocity', data=df_melt, palette='Paired', hue='Fish_label', legend=False, ax=ax)
    ax.tick_params(axis='x', labelsize=16, rotation=45)
    ax.tick_params(axis='y', labelsize=16)
    ax.set_title(f'Velocity Distribution (Box) of All Fish ({TIME_RANGE_START}s to {TIME_RANGE_END}s)', fontsize=24, fontweight='bold')
    ax.set_xlabel('Fish', fontsize=20, fontweight='bold')
    ax.set_ylabel('Speed (cm/s)', fontsize=20, fontweight='bold')
    ax.grid(True, linestyle='--', alpha=0.6)
    # No Y-limit here, to allow the box plot to show the full range including outliers
    ax.set_ylim(-0.8, 5)  # Zoom in on the bulk of the data, excluding extreme outliers. Adjust as needed.

    plt.savefig(f'boxplot_velocity_all_{TIME_RANGE_START}_{TIME_RANGE_END}.png', dpi=DPI, bbox_inches='tight')
    plt.close(fig)

    # ---- Histogram of Averag Speed ----
    fig, ax = plt.subplots(figsize=(12, 8))
    for i in range(1, NUM_FISH + 1):
        speed_col = f'speed{i}'
        if speed_col in df_time_range and not df_time_range[speed_col].isnull().all():
            sns.histplot(df_time_range[speed_col].dropna(), bins=30, kde=True, label=fish_name_mapping[i], ax=ax)
    ax.set_title(f'Histogram of Average Speed of All Fish ({TIME_RANGE_START}s to {TIME_RANGE_END}s)', fontsize=24, fontweight='bold')
    ax.set_xlabel('Speed (cm/s)', fontsize=20, fontweight='bold')
    ax.set_ylabel('Frequency', fontsize=20, fontweight='bold')
    ax.legend(title='Fish', fontsize=16, title_fontsize='18')
    ax.grid(True, linestyle='--', alpha=0.6)
    plt.savefig(f'histogram_speed_all_{TIME_RANGE_START}_{TIME_RANGE_END}.png', dpi=DPI, bbox_inches='tight')
    plt.close(fig)  # Close the figure to free memory

    # --- Barplot of average speed ---\
    fig, ax = plt.subplots(figsize=(12, 8))
    avg_speeds = df_time_range[[f'speed{i}' for i in range(1, NUM_FISH + 1)]].mean()
    avg_speeds.index = [fish_name_mapping[i] for i in range(1, NUM_FISH + 1)]
    avg_speeds.plot(kind='bar', ax=ax, color='skyblue')
    ax.set_title(f'Average Speed of All Fish ({TIME_RANGE_START}s to {TIME_RANGE_END}s)', fontsize=24, fontweight='bold')
    ax.set_xlabel('Fish', fontsize=20, fontweight='bold')
    ax.set_ylabel('Average Speed (cm/s)', fontsize=20, fontweight='bold')
    ax.tick_params(axis='x', rotation=45, labelsize=16)
    ax.tick_params(axis='y', labelsize=16)
    ax.grid(True, linestyle='--', alpha=0.6)
    plt.savefig(f'barplot_avg_speed_all_{TIME_RANGE_START}_{TIME_RANGE_END}.png', dpi=DPI, bbox_inches='tight')
    plt.close(fig)  # Close the figure to free memory


    # 6. Statistical Tests (Combined Data)
    print("Performing statistical tests...")
    velocities = [df_time_range[f'speed{i}'].dropna() for i in range(1, NUM_FISH + 1)]

    # Remove any empty lists of velocities
    velocities = [vel for vel in velocities if not vel.empty]

    # Statistical Tests for Distances
    stats_results_dist = {}
    if len(distances) >= 2:
        f_value_dist, p_value_dist = stats.f_oneway(*distances)
        stats_results_dist['ANOVA F-value'] = f_value_dist
        stats_results_dist['ANOVA p-value'] = p_value_dist
        print(f'ANOVA F-value (Distance): {f_value_dist}, p-value: {p_value_dist}')
    else:
         print("ANOVA on distance not performed: Less than two fish available.")

    if len(distances) >= 2:
        kruskal_h_dist, kruskal_p_dist = stats.kruskal(*distances)
        stats_results_dist['Kruskal-Wallis H-statistic'] = kruskal_h_dist
        stats_results_dist['Kruskal-Wallis p-value'] = kruskal_p_dist
        print(f'Kruskal-Wallis H-statistic (Distance): {kruskal_h_dist}, p-value: {kruskal_p_dist}')
    else:
         print("Kruskal-Wallis on distance not performed: Less than two fish available.")

    stats_df_dist = pd.DataFrame([stats_results_dist])
    stats_df_dist.to_csv(f'stats_distances_{TIME_RANGE_START}_{TIME_RANGE_END}.csv', index=False)

    # Statistical Tests for Velocities
    stats_results_vel = {}
    if len(velocities) >= 2:
       f_value_vel, p_value_vel = stats.f_oneway(*velocities)
       stats_results_vel['ANOVA F-value'] = f_value_vel
       stats_results_vel['ANOVA p-value'] = p_value_vel
       print(f'ANOVA F-value (Velocity): {f_value_vel}, p-value: {p_value_vel}')
    else:
       print("ANOVA on velocity not performed: Less than two fish available.")
    if len(velocities) >= 2:
        kruskal_h_vel, kruskal_p_vel = stats.kruskal(*velocities)
        stats_results_vel['Kruskal-Wallis H-statistic'] = kruskal_h_vel
        stats_results_vel['Kruskal-Wallis p-value'] = kruskal_p_vel
        print(f'Kruskal-Wallis H-statistic (Velocity): {kruskal_h_vel}, p-value: {kruskal_p_vel}')
    else:
      print("Kruskal-Wallis on velocity not performed: Less than two fish available.")

    stats_df_vel = pd.DataFrame([stats_results_vel])
    stats_df_vel.to_csv(f'stats_velocities_{TIME_RANGE_START}_{TIME_RANGE_END}.csv', index=False)

    # Save distance traveled data to CSV
    df_distances = pd.DataFrame({'Fish': labels_with_ids, 'Distance': distances})
    df_distances.to_csv(f'distances_{TIME_RANGE_START}_{TIME_RANGE_END}.csv', index = False)

    # Save speed data to CSV
    df_speeds = df_time_range[[f'speed{i}' for i in range(1, NUM_FISH+1)]].copy()
    df_speeds.rename(columns = {f'speed{i}': fish_name_mapping[i] for i in range(1, NUM_FISH+1)}, inplace = True)
    df_speeds.to_csv(f'speeds_{TIME_RANGE_START}_{TIME_RANGE_END}.csv', index=False)

    # Print summary statistics
    print(f"Summary Statistics ({TIME_RANGE_START}s to {TIME_RANGE_END}s):")
    print(f"Total frames: {len(df_time_range)}")
    print(f"Analyzed time: {df_time_range['time'].max() - df_time_range['time'].min():.2f} seconds")
    print("\nDistance Travelled:")
    for i, dist in enumerate(distances, 1):
        print(f"{labels_with_ids[i-1]}: {dist:.2f} cm")
    print("\nAverage Speed:")
    for i in range(1, NUM_FISH + 1):
       # Ensure speed column exists and is not all NaN before trying to access it
       speed_col = f'speed{i}'
       if speed_col in df_time_range and not df_time_range[speed_col].isnull().all():
           print(f"{fish_name_mapping[i]} (Fish {i}): {df_time_range[speed_col].mean():.2f} cm/s")
    print("\nAnalysis complete.")

## 6. Calling the Analysis Function

This section calls the analysis function with the specified parameters.
**Explanation:**

*   Sets the values for start and end time, and calls the `analyze_zebrafish` function with those parameters.

*  Run the script using `Run All` button at `Top` of this notebook.


In [181]:
# Get analysis range from user (can be hardcoded for testing)
analysis_start_time = 0  # Start time in seconds
analysis_end_time = 120  # End time in seconds
analyze_zebrafish(analysis_start_time, analysis_end_time)

Analyzing data from 0s to 120s...
Generating combined trajectory plot...
Generating individual density plots...


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13868\1559771407.py:46: UserWarning: KDE cannot be estimated (0 variance or perfect covariance). Pass `warn_singular=False` to disable this warning.
  sns.kdeplot(x=df_normalized[x_col].dropna(), y=df_normalized[y_col].dropna(),
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13868\1559771407.py:46: UserWarning: KDE cannot be estimated (0 variance or perfect covariance). Pass `warn_singular=False` to disable this warning.
  sns.kdeplot(x=df_normalized[x_col].dropna(), y=df_normalized[y_col].dropna(),


Generating distance travelled bar plot...
Distance travelled by Control (Fish 1): 82.94 cm
Distance travelled by Control (Fish 2): 96.71 cm
Distance travelled by 0.08 mg/mL (Fish 3): 96.29 cm
Distance travelled by 0.08 mg/mL (Fish 4): 59.91 cm
Distance travelled by 0.16 mg/mL (Fish 5): 58.02 cm
Distance travelled by 0.16mg/mL (Fish 6): 8.47 cm
Distance travelled by 0.24 mg/mL (Fish 7): 35.52 cm
Distance travelled by 0.24 mg/mL (Fish 8): 32.64 cm
Distance travelled by 0.32 mg/mL (Fish 9): 66.75 cm
Distance travelled by 0.32 mg/mL (Fish 10): 24.96 cm
Distance travelled by 3.6 mg/mL (Fish 11): 46.72 cm
Distance travelled by 3.6 mg/mL (Fish 12): 64.06 cm
Distance travelled by 4.4 mg/mL (Fish 13): 7.11 cm
Distance travelled by 4.4 mg/mL (Fish 14): 64.54 cm
Distance travelled by 5.2 mg/mL (Fish 15): 43.69 cm
Distance travelled by 5.2 mg/mL (Fish 16): 17.18 cm
Distance travelled by 6 mg/mL (Fish 17): 49.75 cm
Distance travelled by 6 mg/mL (Fish 18): 54.84 cm
Generating individual speed plots.

C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\scipy\stats\_axis_nan_policy.py:580: SmallSampleWarning: all input arrays have length 1.  f_oneway requires that at least one input has length greater than 1.
  if is_too_small(samples, kwds):
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13868\1559771407.py:194: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  f_value_dist, p_value_dist = stats.f_oneway(*distances)
